### This project is basically an overview of how we can train a machine learning model on the diabeties data set and predict with the conditions given whether the person is diabetic or not.






## SVM
### We use svm classification for this project
Support vector machine used to classify a data point based on a maximum margin hyperplane created by the supporting points known as support vectors (vectors - as in higher dimension it is tough to call them points, supporting - since these points are enough than all the other data points together as this marks the boundary to. help classify the point) which seperate the data points.

Let's Start!

About the libraries

1.   numpy - for numpy arrays, useful for processing and scientific computing 
2.   pandas - helpful for creating dataframes and storing data
3.   StandardScalar - in this project we're using support vector machine classification and this class cannot process the data given to it unless the data is standardized. 
4.   svm - the suport vector machine class in the sklearn package
5.   accuracy_score - to check the accuracy of our model
6.   train_test_split - to split the data into training and test set




Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


Data Collecting and Analysis


*   dataset - PIMA Indians diabetes dataset (Kaggle) 
    this data set contains data about females 



In [ ]:
diabetes_dataset = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
diabetes_dataset.head()

In [ ]:
diabetes_dataset.describe()

In [ ]:
diabetes_dataset.shape

Let's visualize the number of

* 0 --> Not diabetic
* 1 --> Diabetic

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(x=diabetes_dataset['Outcome'])
plt.title('Outcome vs count',fontsize=20)
plt.xlabel('Outcome',fontsize=15)
plt.ylabel('Count',fontsize=15);

In [ ]:
diabetes_dataset["Outcome"].value_counts()

In [ ]:
diabetes_dataset.groupby('Outcome').mean()

In [ ]:
X = diabetes_dataset.iloc[:,:-1].values
y = diabetes_dataset.iloc[:,-1].values

In [ ]:
X

In [ ]:
y

Data Standardization


In [ ]:
scalar = StandardScaler()
X = scalar.fit_transform(X)
print(X)

Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, stratify = y)

In [ ]:
X_train.shape
y_train.shape

Training the model

In [ ]:
classifier = svm.SVC(C=0.5, kernel='linear')
#classifier = LogisticRegression()
#classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski' , p=2)
#classifier = svm.SVC(kernel = 'rbf')
#classifier = GaussianNB()
#classifier = DecisionTreeClassifier(criterion='entropy')
#classifier = RandomForestClassifier(n_estimators=10,criterion = 'entropy')
classifier.fit(X_train,y_train)

Model Evaluation
Accuracy Score

In [ ]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_pred,y_test)
accuracy = accuracy_score(y_pred, y_test)
print(cm)
cf_matrix = cm
print("Accuracy of the model is:", accuracy)

Using K-Fold cross validation to check if we didn't get lucky on the test set!

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier,X = X_train,y= y_train , cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f}".format(accuracies.std()*100))

In [ ]:
#.  visualizing the confusion matrix!
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Reds')

In [ ]:
#Visualzing. with labels!
labels = ['True Neg','False Pos','False Neg','True Pos']
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Reds')

Now we have to check if there is a better hyperparameter we can tune to improve over all accuracy.
We use **Grid Search** here

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[0.25,0.5,0.75,1], 'kernel' : ['linear']},
              {'C':[0.25,0.5,0.75,1], 'kernel' : ['rbf'], 'gamma' : [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}]
grid_search = GridSearchCV(estimator=classifier,
                          param_grid=parameters,
                          scoring='accuracy',
                          cv=10)
grid_search.fit(X_train,y_train)
print("Best Accuracy: {:.2f} %".format(grid_search.best_score_*100))
print("Best Parameters: ", grid_search.best_params_)

Accuracy comparision of all classifier methods

1.   SVM - using svm we get a accuracy of 78% and as we can see the best hyperparameters to use is c=0.5 and linear kernel
2.   Logistic Regression - accuracy of 78%
3.   KNN - accuracy of 74%
4.   Kernel SVM - accuracy of 77%
5.   Naive Bayes - accuracy of 74%
6.   Decision Tree - accuracy of 71%
7.   Random Forest - accuracy of 78% though random forest has good accuracy it performs too good in train set and is over fitted



Making a predictive system

In [ ]:
input_data = np.array([1,85,66,29,0,26.6,0.351,31])
input_data = input_data.reshape(1,-1)
standerdized_input_data = scalar.transform(input_data)
prediction = classifier.predict(standerdized_input_data)
if(prediction[0] == 0):
  print("Person is not diabetic")
else:
  print("The Person is diabetic")